In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import PIL

In [ ]:

    
# import PIL.Image


# def dataset(file_path, img_width, img_height):
#     X = []
#     y = []
#     folder = os.listdir(file_path)
#     folder.remove('.DS_Store') 
#     for i in folder:
#         each_folder = os.path.join(file_path, i)
#         each_image = os.listdir(each_folder)
       
#         for j in each_image:
#             img = os.path.join(each_folder, j)
#             img = PIL.Image.open(img)
#             img.resize((img_width, img_height))
#             img = np.asarray(img)
#             X.append(img)
#             y.append(i)
            
        
        

#     return X,y

# file_path = "/Users/ememusoh/Library/CloudStorage/OneDrive-Personal/Baby Love/ml_tutorial/Machine Learning Projects/dataset"
# # X,y = dataset(file_path)
# # print(len(X))
# # print(len(y))
# X,y = dataset(file_path, 256, 256)
# print(X)


In [ ]:
def images(file_path, width, height):
    X = []
    y = []
    folder = os.listdir(file_path)
    folder.remove('.DS_Store')
    for i in folder:
        each_folder_path = os.path.join(file_path, i)
        each_folder = os.listdir(each_folder_path)
        for j in each_folder:
            img = os.path.join(each_folder_path, j)
            img = PIL.Image.open(img)
            img = img.resize((width, height))
            img = np.asarray(img)
            X.append(img)
            y.append(i)
    return X,y
file_path = "/Users/ememusoh/Library/CloudStorage/OneDrive-Personal/Baby Love/ml_tutorial/Machine Learning Projects/dataset"
X,y = images(file_path, 299, 299)
X = np.stack(X, axis=0)
X = X/255.0
y = np.asarray(y)
print(X.shape)
print(y.shape)
    

In [ ]:
#We have to onehot encode the y

import sklearn 
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_encode = encoder.fit_transform(y)

print(f"the orginal:", y)
print(f"the encode_y:" ,y_encode)

In [ ]:
## splt data into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_encode, test_size=0.2, random_state=42, stratify=y)
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)


print(x_train.shape)
print(X_test.shape) 
print(x_val.shape) 

In [ ]:
X_train.dtype

In [85]:
# import tensorflow as tf
# train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# val_ds   = tf.data.Dataset.from_tensor_slices((x_val,   y_val))
# test_ds  = tf.data.Dataset.from_tensor_slices((X_test,  y_test))
# BATCH_SIZE=32
# train_ds = (
#         train_ds
#         .shuffle(buffer_size=1000)   # Shuffle the dataset
        
#         .batch(BATCH_SIZE)
#         .prefetch(tf.data.AUTOTUNE)
#         )

# # For val_ds and test_ds, we typically don't do augmentation; just batch & prefetch
# val_ds = (
#         val_ds
#         .batch(BATCH_SIZE)
#         .prefetch(tf.data.AUTOTUNE)
#         )

# test_ds = (
#         test_ds
        
#         .batch(BATCH_SIZE)
#         .prefetch(tf.data.AUTOTUNE)
#         )


# for image, label in train_ds.take(2):
#     print(image.shape)
#     print(label.numpy())

In [ ]:
plt.imshow(image)

In [86]:

import tensorflow as tf
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_ds   = tf.data.Dataset.from_tensor_slices((x_val,   y_val))
test_ds  = tf.data.Dataset.from_tensor_slices((X_test,  y_test))

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),      # flips
    tf.keras.layers.RandomRotation(0.1),           # slight rotation
    tf.keras.layers.RandomZoom(0.1),               # zoom in/out

    # Add more if needed, e.g. RandomContrast, RandomTranslation...
])

def augment(image, label):
    # Convert image to float32 if not already
    image = tf.cast(image, tf.float32)
    
    # Apply data augmentation
    image = data_augmentation(image)
    return image, label
def convert_to_float32(image, label):
    image = tf.cast(image, tf.float32)
    return image, label
    

# Shuffle, batch, and prefetch the training set
BATCH_SIZE = 32

train_ds = (
    train_ds.shuffle(buffer_size=1000)   # Shuffle the dataset
    .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

# For val_ds and test_ds, we typically don't do augmentation; just batch & prefetch
val_ds = (
    val_ds
    .map(convert_to_float32, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    test_ds
    .map(convert_to_float32, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)


In [89]:
for image, label in train_ds.take(1):
    print(image.shape)
    print(label.numpy())

(32, 299, 299, 3)
[6 0 0 9 1 4 3 6 7 9 4 8 2 1 4 6 3 3 1 2 5 7 8 2 0 9 9 3 1 4 1 5]


2025-02-03 22:44:25.106775: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [91]:
count = 0
for image, label in train_ds:
    count += len(image)
    

print("Total size:", count)

Total size: 618


2025-02-03 22:47:39.714652: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [93]:
## create the model

import tensorflow as tf
from tensorflow.keras import layers, models

def conv_bn_relu(x, filters, kernel_size, strides):
    x = layers.Conv2D(filters, kernel_size, strides= strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.2)(x)
    return x

def sep_conv_bn_relu(x, filters, kernel_size):
    x = layers.SeparableConv2D(filters, kernel_size, strides = 1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.2)(x)
    return x

def xception_block(x,in_filters, out_filters):
    res = layers.Conv2D(out_filters, 1, strides= 2, padding='same')(x)
    res = layers.BatchNormalization()(res)
    res = layers.Dropout(0.2)(res)
    
    x = sep_conv_bn_relu(x, in_filters, 3)
    x = sep_conv_bn_relu(x, in_filters, 3)
    x = layers.MaxPool2D(pool_size =3, strides =2, padding='same')(x)
    
    x = layers.Concatenate()([x, res])
    return x


def Xception(input_shape, num_classes):
    input = layers.Input(shape=input_shape)
    x = conv_bn_relu(input, 12, 3, 2)
    x = conv_bn_relu(x, 24, 3, 1)
    
    x = xception_block(x, in_filters=12, out_filters=12)
    x = xception_block(x, in_filters=24, out_filters=24)
    x = xception_block(x, in_filters=32, out_filters=32)
    
    for i in range(5):
        res = x
        x = sep_conv_bn_relu(x, 32, 3)
        x = sep_conv_bn_relu(x, 32, 3)
        x = sep_conv_bn_relu(x, 32, 3)
        x = layers.Concatenate()([x, res])
        
    x = xception_block(x, 12, 32)
    
    x = sep_conv_bn_relu(x, 12, 3)
    x = sep_conv_bn_relu(x, 24, 3)
    
    x = layers.GlobalAvgPool2D()(x)
    output = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(input, output)
    return model

input_shape=(299,299,3)

model = Xception(input_shape, 10)

In [94]:
model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_16      │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_75 (Conv2D)  │ (None, 150, 150,  │        336 │ input_layer_16[0… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 150, 150,  │         48 │ conv2d_75[0][0]   │
│ (BatchNormalizatio… │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_369 (ReLU)    │ (None, 150, 150,  │          0 │ batch_normalizat… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_213         │ (None, 150, 150,  │          0 │ re_lu_369[0][0]   │
│ (Dropout)           │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_76 (Conv2D)  │ (None, 150, 150,  │      2,616 │ dropout_213[0][0] │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 150, 150,  │         96 │ conv2d_76[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_370 (ReLU)    │ (None, 150, 150,  │          0 │ batch_normalizat… │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_214         │ (None, 150, 150,  │          0 │ re_lu_370[0][0]   │
│ (Dropout)           │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_3… │ (None, 150, 150,  │        516 │ dropout_214[0][0] │
│ (SeparableConv2D)   │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 150, 150,  │         48 │ separable_conv2d… │
│ (BatchNormalizatio… │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_371 (ReLU)    │ (None, 150, 150,  │          0 │ batch_normalizat… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_216         │ (None, 150, 150,  │          0 │ re_lu_371[0][0]   │
│ (Dropout)           │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_3… │ (None, 150, 150,  │        264 │ dropout_216[0][0] │
│ (SeparableConv2D)   │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 150, 150,  │         48 │ separable_conv2d… │
│ (BatchNormalizatio… │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_372 (ReLU)    │ (None, 150, 150,  │          0 │ batch_normalizat… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_77 (Conv2D)  │ (None, 75, 75,    │        300 │ dropout_214[0][0

 Total params: 68,050 (265.82 KB)

 Trainable params: 66,426 (259.48 KB)

 Non-trainable params: 1,624 (6.34 KB)

In [96]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3), loss= "sparse_categorical_crossentropy", metrics = ["accuracy"])


In [97]:
history = model.fit(train_ds, epochs=150, validation_data=val_ds)

Epoch 1/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 46s 658ms/step - accuracy: 0.1118 - loss: 2.3544 - val_accuracy: 0.0968 - val_loss: 2.3024
Epoch 2/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 374ms/step - accuracy: 0.1352 - loss: 2.2721 - val_accuracy: 0.0968 - val_loss: 2.3026
Epoch 3/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 377ms/step - accuracy: 0.1421 - loss: 2.2637 - val_accuracy: 0.0968 - val_loss: 2.3031
Epoch 4/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 360ms/step - accuracy: 0.1859 - loss: 2.2043 - val_accuracy: 0.0968 - val_loss: 2.3042
Epoch 5/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 393ms/step - accuracy: 0.2038 - loss: 2.1451 - val_accuracy: 0.0968 - val_loss: 2.3062
Epoch 6/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 363ms/step - accuracy: 0.2331 - loss: 2.1581 - val_accuracy: 0.0968 - val_loss: 2.3096
Epoch 7/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 357ms/step - accuracy: 0.2595 - loss: 2.0684 - val_accuracy: 0.0968 - val_loss: 2.3144
Epoch 8/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 367ms/step - accuracy: 0.2347 - loss: 2.0842 - val_acc

There are different ways to reduce overfitting: (we call it regularization).add dropout, change learning rate, reduce the achitecture(filters), data_agumentation

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))